In [25]:
import bayes3d as b
import numpy as np 
import jax.numpy as jnp
import matplotlib.pyplot as plt
import jax
import PIL.Image
from math import sqrt
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML

In [23]:
def display_video(frames, framerate=30):
    if type(frames[0]) == PIL.Image.Image:
      frames = [np.array(frames[i]) for i in range(len(depths))]
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

In [5]:
intrinsics = b.Intrinsics(
    height=360,
    width=480,
    fx=180*sqrt(3), fy=180*sqrt(3),
    cx=240.0, cy=180.0,
    near=0.1, far=100.0
)

In [6]:
b.setup_renderer(intrinsics)
# b.RENDERER.add_mesh_from_file("assets/cube.obj",mesh_name="cube_0", scaling_factor=0.5)

Increasing frame buffer size to (width, height, depth) = (480, 384, 1024)


[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


In [7]:
b.RENDERER.add_mesh_from_file("assets/cube.obj",mesh_name="cube_1", scaling_factor=0.5)

In [8]:
loaded_poses = np.load('poses_camera_frame.npz')

In [9]:
loaded_box_poses = loaded_poses['box']
gt_poses = []
for i in range(loaded_box_poses.shape[0]):
    pose = loaded_box_poses[i,...]
    pose[1:3] *= -1 # convert to CV2 convention (bayes3d currently assumes input is in cv2 coordinate frame)
    gt_poses.append(jnp.array(pose))
gt_poses = jnp.stack(gt_poses)

In [10]:
gt_images = jax.vmap(b.RENDERER.render_single_object, in_axes=(0, None))(gt_poses, jnp.int32(0))

In [11]:
depths = [b.viz.get_depth_image(gt_images[i,:,:,2]) for i in range(gt_images.shape[0])]
b.make_gif(depths, "test.gif")

In [17]:
type(depths[0])

PIL.Image.Image

In [26]:
display_video(depths)